In [ ]:
# Read in csv
# Clean data sheet
# Matching algo

In [92]:
# Math, stat and data
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm

## Data Exploration

###### Concentrate Rate = Number of orders within 1 hour / Number of Unique Buyers within 1 hour

In [6]:
base = pd.read_csv('order_brush_order.csv')

In [29]:
data_set = base
data_set['event_time'] = pd.to_datetime(base['event_time'])
data_set['shopid'] = pd.to_numeric(base['shopid'])
data_set['userid'] = pd.to_numeric(base['userid'])

In [63]:
ordered_data = data_set.sort_values(['shopid','event_time'], ascending = (True))

In [64]:
ordered_data = ordered_data.reset_index()
ordered_data = ordered_data.drop(['index'], axis = 1)

In [65]:
ordered_data['buyers_in_next_hour'] = ''

In [66]:
ordered_data['no_orders_in_next_hour'] = 1

In [93]:
one_hour = datetime.timedelta(hours = 1)

for i in tqdm(range(len(ordered_data))):
    current_id = ordered_data['shopid'][i]
    buyers = [ordered_data['userid'][i]]
    start_time = ordered_data['event_time'][i]
    
    for j in range(i+1,len(ordered_data)):
        if current_id == ordered_data['shopid'][j]:
            if (ordered_data['event_time'][j] - start_time) < one_hour:
                buyers.append(ordered_data['userid'][j])
            else:
                break
        else:
            break
            
    ordered_data['buyers_in_next_hour'][i] = buyers

In [172]:
ordered_data

,orderid,shopid,userid,event_time,buyers_in_next_hour,no_orders_in_next_hour
0,31086409141107,10009,196962305,2019-12-27 03:06:50,[196962305],1
1,31144571933461,10051,2854032,2019-12-27 19:16:11,[2854032],1
2,31254979546679,10051,48600461,2019-12-29 01:56:19,[48600461],1
3,31195675919209,10061,168750452,2019-12-28 09:27:55,[168750452],1
4,31205132327893,10061,194819216,2019-12-28 12:05:32,[194819216],1
...,...,...,...,...,...,...
222745,31493203163305,214949521,46269178,2019-12-31 20:06:43,[46269178],1
222746,31328775676314,214964814,200983383,2019-12-29 22:26:16,[200983383],1
222747,31453591917585,215175775,13688804,2019-12-31 09:06:31,[13688804],1
222748,31472077221692,215175775,129266028,2019-12-31 14:14:37,[129266028],1


In [189]:
ordered_data = completed_data.copy()

In [234]:
def calc_no_buyers(row):
    return len(row['buyers_in_next_hour'])

def unique_buyers(row):
    return np.unique(row['buyers_in_next_hour'])

def no_unique_buyers(row):
    return len(row['unique_buyers_in_next_hour'])

def culprit(row):
    if row['order_brush'] == 1:
        return max(set(row['buyers_in_next_hour']), key = row['buyers_in_next_hour'].count)
    else:
        return -1

In [175]:
completed_data['no_orders_in_next_hour'] = completed_data.apply(calc_no_buyers, axis = 1)
completed_data['unique_buyers_in_next_hour'] = completed_data.apply(unique_buyers, axis = 1)
completed_data['no_unique_buyers_in_next_hour'] = completed_data.apply(no_unique_buyers, axis = 1)
completed_data['concentrate_rate'] = completed_data['no_orders_in_next_hour']/completed_data['no_unique_buyers_in_next_hour']
completed_data['order_brush'] = (completed_data['concentrate_rate'] >= 3).astype(int)

In [235]:
completed_data['culprit'] = completed_data.apply(culprit, axis = 1)

In [284]:
completed_data

,orderid,shopid,userid,event_time,buyers_in_next_hour,no_orders_in_next_hour,unique_buyers_in_next_hour,no_unique_buyers_in_next_hour,culprit,concentrate_rate,order_brush
0,31086409141107,10009,196962305,2019-12-27 03:06:50,[196962305],1,[196962305],1,-1,1.0,0
1,31144571933461,10051,2854032,2019-12-27 19:16:11,[2854032],1,[2854032],1,-1,1.0,0
2,31254979546679,10051,48600461,2019-12-29 01:56:19,[48600461],1,[48600461],1,-1,1.0,0
3,31195675919209,10061,168750452,2019-12-28 09:27:55,[168750452],1,[168750452],1,-1,1.0,0
4,31205132327893,10061,194819216,2019-12-28 12:05:32,[194819216],1,[194819216],1,-1,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...
222745,31493203163305,214949521,46269178,2019-12-31 20:06:43,[46269178],1,[46269178],1,-1,1.0,0
222746,31328775676314,214964814,200983383,2019-12-29 22:26:16,[200983383],1,[200983383],1,-1,1.0,0
222747,31453591917585,215175775,13688804,2019-12-31 09:06:31,[13688804],1,[13688804],1,-1,1.0,0
222748,31472077221692,215175775,129266028,2019-12-31 14:14:37,[129266028],1,[129266028],1,-1,1.0,0


In [237]:
df = completed_data.copy()
df['culprit'] = df['culprit'].astype(str)

In [238]:
df['culprit'] = df[['shopid','culprit']].groupby(['shopid'])['culprit'].transform(lambda x: '&'.join(x))

# df_temp = df[['shopid','culprit']].groupby(['shopid'])['culprit']
# def concat(data):
#     return '&'.join(data)
# df['culprit'] = df_temp['culprit'].apply(concat)

submission = df[['shopid','culprit']].drop_duplicates()

In [241]:
def replacer(row):
    return row['culprit'].replace("-1&","")

def replacer2(row):
    return row['userid_1'].replace("-1","")

In [242]:
submission['userid_1'] = submission.apply(replacer, axis = 1)
submission['userid'] = submission.apply(replacer2, axis = 1)

In [278]:
final = submission.drop(['culprit', 'userid_1'], axis = 1)
final = final.reset_index()

In [279]:
def right_strip(row):
    return row['userid'].rstrip('&')

In [280]:
final['userid'] = final.apply(right_strip, axis = 1)

In [281]:
for i in tqdm(range(len(final))):
    if final['userid'][i] == "":
        final['userid'][i] = "0"

In [282]:
final

,index,shopid,userid
0,0,10009,0
1,1,10051,0
2,3,10061,0
3,7,10084,0
4,62,10100,0
...,...,...,...
18765,222743,214662358,0
18766,222744,214949521,0
18767,222746,214964814,0
18768,222747,215175775,0


In [283]:
final.to_csv('Lets_go_Makan_3.csv', index = False)